In [6]:
import pandas as pd
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [7]:
!pip show pandas

Name: pandas
Version: 1.4.0
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: 


all
- org count
- volume sum
- trans count
- max amount
- mean amount
- new recurring count

segment
- org count
- volume sum
- amount mean
- amount median

In [8]:
hours = [9, 13, 15]
hour = 15

gt_2019 = ["date='2019-12-03' or (date='2019-12-04' and hour<3)"]
gt_2020 = ["date='2020-12-01' or (date='2020-12-02' and hour<3)"]
gt_2021 = ["date='2021-11-30' or (date='2021-12-01' and hour<3)"]
gt_2022 = ["date='2022-11-29' or (date='2022-11-30' and hour<3)"]

## all transactions

In [9]:
q = '''select
            count(distinct(org)) as orgs,
            sum(amount) as volume,
            count(distinct(id)) as trans_count,
            max(amount) as max_amount,
            avg(amount) as avg_amount
        from transactions
        where status='A' and recurring=0 and '''
q = '''select
            count(distinct(org)) as orgs,
            sum(amount) as volume,
            count(distinct(id)) as trans_count,
            max(amount) as max_amount,
            avg(amount) as avg_amount
        from transactions
        where status='A' and '''

In [10]:
trans_2019 = redshift_query_read(q + ' and '.join(gt_2019), schema='production')
trans_2019['year'] = 2019
trans_2020 = redshift_query_read(q + ' and '.join(gt_2020), schema='production')
trans_2020['year'] = 2020
trans_2021 = redshift_query_read(q + ' and '.join(gt_2021), schema='production')
trans_2021['year'] = 2021
trans_2022 = redshift_query_read(q + ' and '.join(gt_2022), schema='production')
trans_2022['year'] = 2022

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/AftonCoombs/Documents/Recommendation

In [11]:
all_stats = trans_2019.append(trans_2020).append(trans_2021).append(trans_2022)

/var/folders/3s/61ynz9t96wn0xs3dbklz1ln00000gp/T/ipykernel_45230/1411044503.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_stats = trans_2019.append(trans_2020).append(trans_2021).append(trans_2022)
/var/folders/3s/61ynz9t96wn0xs3dbklz1ln00000gp/T/ipykernel_45230/1411044503.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_stats = trans_2019.append(trans_2020).append(trans_2021).append(trans_2022)


In [12]:
print("| year | orgs | volume | transactions | max amount | average amount |")
for _, r in all_stats.sort_values('year', ascending=True).iterrows():
    print("| {} | {:} | ${:.2f} | {:} | ${:.2f} | ${:.2f} |".format(r['year'], r['orgs'], r['volume'], r['trans_count'], r['max_amount'], r['avg_amount']))

| year | orgs | volume | transactions | max amount | average amount |
| 2019.0 | 1331.0 | $3131006.10 | 20906.0 | $50000.00 | $149.77 |
| 2020.0 | 1716.0 | $6489235.18 | 41429.0 | $100000.00 | $156.64 |
| 2021.0 | 1836.0 | $6687216.41 | 42241.0 | $46350.00 | $158.31 |
| 2022.0 | 1937.0 | $6563590.33 | 43110.0 | $50000.00 | $152.25 |


## recurring

In [13]:
gt_2020 = ["date='2020-12-01'"]
gt_2021 = ["date='2021-11-30'"]
gt_2022 = ["date='2022-11-29'"]

q = '''select
            count(recurring)
        from (select
                recurring,
                min(date) as date
            from transactions
            where
                status='A' and
                recurring!=0 and
                {}
            group by recurring) as rec
        where {}'''

#rec_2019 = redshift_query_read(q + ' and '.join(gt_2019), schema='production')
#rec_2019['year'] = 2019
rec_2020 = redshift_query_read(q.format(gt_2020[0], gt_2020[0]), schema='production')
rec_2020['year'] = 2020
rec_2021 = redshift_query_read(q.format(gt_2021[0], gt_2021[0]), schema='production')
rec_2021['year'] = 2021
rec_2022 = redshift_query_read(q.format(gt_2022[0], gt_2022[0]), schema='production')
rec_2022['year'] = 2022

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
rec_2020.append(rec_2021).append(rec_2022)

/var/folders/3s/61ynz9t96wn0xs3dbklz1ln00000gp/T/ipykernel_45230/1935790890.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rec_2020.append(rec_2021).append(rec_2022)


,count,year
0,5840,2020
0,8425,2021
0,861,2022


In [15]:
q = '''select
            recurring, 
            min(date) as min_date,
            count(id) as occurrences
        from transactions
        where
            status='A' and
            recurring!=0
        group by recurring'''
rec = redshift_query_read(q, schema='production')

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
rec['min_date'] = pd.to_datetime(rec['min_date'])

rec['gt_2020'] = rec['min_date']=='2020-12-01'
rec['gt_2021'] = rec['min_date']=='2021-11-30'
rec['gt_2022'] = rec['min_date']=='2022-11-29'

In [17]:
print("2020: {}".format(rec['gt_2020'].value_counts()[1]))
print("2021: {}".format(rec['gt_2021'].value_counts()[1]))
print("2022: {}".format(rec['gt_2022'].value_counts()[1]))

2020: 786
2021: 692
2022: 711


# segments

In [18]:
q = "select id, segment from organization"
orgs = redshift_query_read(q, schema='production')

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
q = '''select
            id,
            org,
            amount
        from transactions
        where
            status='A' and ''' + gt_2022[0]
trans_2022 = redshift_query_read(q, schema='production')

/Users/AftonCoombs/Documents/Recommendation-master/.venv/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
org_trans = trans_2022.merge(orgs, left_on='org', right_on='id')

segs_stats = org_trans.groupby('segment')['amount'].agg(['count', 'sum', 'mean', 'median']).reset_index()

In [22]:
segs_stats

,segment,count,sum,mean,median
0,A - Arts; Culture; and Humanities,1072,163734.68,152.737575,52.500
1,B - Educational Institutions,3087,559296.39,181.177969,52.500
2,C - Environmental Advocacy and Protection,1180,166704.07,141.274636,52.000
3,D - Animal,4301,441252.01,102.592888,51.500
4,E - Health; General and Rehabilitative,874,173786.66,198.840572,52.750
5,F - Mental Health and Crisis Intervention,485,88955.54,183.413485,100.000
6,G - Disease; Disorders; Medical Disciplines,731,132792.15,181.658208,62.400
7,H - Medical Research,442,96790.98,218.984118,100.000
8,I - Crime; Legal Related,159,38492.04,242.088302,100.000
9,J - Employment; Job Related,60,9777.95,162.965833,62.400
